In [ ]:
#Build a naive Bayes model on the data set for classifying the ham and spam


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [2]:
sh_data = pd.read_csv("E:\\Excelr\\Assignments\\Assignments DS\\sms_raw_NB.csv",encoding = "ISO-8859-1")
sh_data.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or å£10,000..."
4,spam,okmail: Dear Dave this is your final notice to...


In [3]:
sh_data.groupby('type').count()

,text
type,
ham,4812
spam,747


In [4]:
"This is Awsome 1231312 $#%$# a i he yu nwj"
import re
def cleaning_text(i):
    i = re.sub("[^A-Za-z" "]+"," ",i).lower()
    i = re.sub("[0-9" "]+"," ",i)
    w = []
    for word in i.split(" "):
        if len(word)>2:
            w.append(word)
    return (" ".join(w))

In [5]:
"This is Awsome 1231312 $#%$# a i he yu nwj".split(" ")

cleaning_text("This is Awsome 1231312 $#%$# a i he yu nwj")

'this awsome nwj'

In [6]:
# testing above function with sample text => removes punctuations, numbers
cleaning_text("Hope you are having a good week. Just checking in")

'hope you are having good week just checking'

In [7]:
sh_data.text = sh_data.text.apply(cleaning_text)

In [8]:
# removing empty rows 
sh_data.shape

(5559, 2)

In [9]:
sh_data = sh_data.loc[sh_data.text != " ",:]

In [10]:
# creating a matrix of token counts for the entire text document 

def split_into_words(i):
    return (i.split(" "))

In [11]:
# splitting data into train and test data sets 
from sklearn.model_selection import train_test_split
sh_train,sh_test = train_test_split(sh_data,test_size=0.3)

In [12]:
# Preparing email texts into word count matrix format 
sh_wo = CountVectorizer(analyzer=split_into_words).fit(sh_data.text)

In [13]:
# For all messages
all_sh_matrix = sh_wo.transform(sh_data.text)
all_sh_matrix.shape 

(5559, 7429)

In [14]:
# For training messages
train_sh_matrix = sh_wo.transform(sh_train.text)
train_sh_matrix.shape

(3891, 7429)

In [15]:
# For testing messages
test_sh_matrix = sh_wo.transform(sh_test.text)
test_sh_matrix.shape

(1668, 7429)

In [16]:
####### Without TFIDF matrices ########################
# Preparing a naive bayes model on training data set 

from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB

In [18]:
# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_sh_matrix,sh_train.type)

MultinomialNB()

In [19]:
#train prediction
train_pred_m = classifier_mb.predict(train_sh_matrix)
train_pred_m

array(['ham', 'spam', 'ham', ..., 'spam', 'ham', 'ham'], dtype='<U4')

In [20]:
accuracy_train_m = np.mean(train_pred_m==sh_train.type)
accuracy_train_m #98.86

0.9886918529940889

In [22]:
#test prediction
test_pred_m = classifier_mb.predict(test_sh_matrix)
test_pred_m

array(['ham', 'ham', 'spam', ..., 'ham', 'spam', 'ham'], dtype='<U4')

In [23]:
accuracy_test_m = np.mean(test_pred_m==sh_test.type)
accuracy_test_m #97.24

0.9724220623501199

In [24]:
# Gaussian Naive Bayes 
classifier_gb = GB()
classifier_gb.fit(train_sh_matrix.toarray(),sh_train.type.values) # we need to convert tfidf into array format which is compatible for gaussian naive bayes

GaussianNB()

In [25]:
#train prediction
train_pred_g = classifier_gb.predict(train_sh_matrix.toarray())
train_pred_g

array(['ham', 'spam', 'ham', ..., 'spam', 'ham', 'ham'], dtype='<U4')

In [26]:
accuracy_train_g = np.mean(train_pred_g==sh_train.type)
accuracy_train_g #93.44

0.9344641480339244

In [27]:
#test prediction
test_pred_g = classifier_gb.predict(test_sh_matrix.toarray())
test_pred_g

array(['ham', 'ham', 'spam', ..., 'ham', 'spam', 'ham'], dtype='<U4')

In [28]:
accuracy_test_g = np.mean(test_pred_g==sh_test.type) 
accuracy_test_g #87.52

0.8752997601918465

In [29]:
# Learning Term weighting and normalizing on entire emails
tfidf_transformer = TfidfTransformer().fit(all_sh_matrix)

In [30]:
# Preparing TFIDF for train emails
train_tfidf = tfidf_transformer.transform(train_sh_matrix)

In [31]:
train_tfidf.shape

(3891, 7429)

In [32]:
# Preparing TFIDF for test emails
test_tfidf = tfidf_transformer.transform(test_sh_matrix)

In [33]:
test_tfidf.shape

(1668, 7429)

In [34]:
# Preparing a naive bayes model on training data set 

from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB

In [35]:
# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_tfidf,sh_train.type)

MultinomialNB()

In [36]:
train_pred_m = classifier_mb.predict(train_tfidf)
train_pred_m

array(['ham', 'spam', 'ham', ..., 'spam', 'ham', 'ham'], dtype='<U4')

In [37]:
accuracy_train_m = np.mean(train_pred_m==sh_train.type)
accuracy_train_m #96.40

0.9640195322539193

In [39]:
test_pred_m = classifier_mb.predict(test_tfidf)
test_pred_m

array(['ham', 'ham', 'spam', ..., 'ham', 'spam', 'ham'], dtype='<U4')

In [40]:
accuracy_test_m = np.mean(test_pred_m==sh_test.type)
accuracy_test_m #96.28

0.9628297362110312

In [41]:
# Gaussian Naive Bayes 
classifier_gb = GB()
classifier_gb.fit(train_tfidf.toarray(),sh_train.type.values)

GaussianNB()

In [42]:
train_pred_g = classifier_gb.predict(train_tfidf.toarray())
train_pred_g 

array(['ham', 'spam', 'ham', ..., 'spam', 'ham', 'ham'], dtype='<U4')

In [43]:
accuracy_train_g = np.mean(train_pred_g==sh_train.type)
accuracy_train_g #93.44

0.9344641480339244

In [44]:
test_pred_g = classifier_gb.predict(test_tfidf.toarray())
test_pred_g 

array(['ham', 'ham', 'spam', ..., 'ham', 'spam', 'ham'], dtype='<U4')

In [45]:
accuracy_test_g = np.mean(test_pred_g==sh_test.type)
accuracy_test_g #87.35% test data is successfully classified.

0.8735011990407674